In [1]:
from geopy.geocoders import Nominatim
import polars as pl
import plotly.express as px
from polars import col as c

In [2]:
df = pl.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSvTrBNbcpx6fu7DBog8kbYiSJOYVwxP1cQHs_bBwiGYYIQglMAaY9Aw_tw4Rn0jswDN0vcAH-DS8A2/pub?output=csv', truncate_ragged_lines=True)

In [3]:
# Remove all the empty rows
df = df.filter(c('Quilt Name') != ' ')

# Get all the latitude and longitudes
geolocator = Nominatim(user_agent="quilt_tracker")
locs = df['Location'].map_elements(geolocator.geocode)
df = df.with_columns(
    lat=locs.map_elements(lambda l: l.latitude, return_dtype=float),
    lon=locs.map_elements(lambda l: l.longitude, return_dtype=float),
    Details=c('Quilt Show/Museum') + ' in ' + c('Location') + '. ' + c('Additional Notes') + ' URL: ' + c('Website')
)

/tmp/ipykernel_596768/2286330391.py:6: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  locs = df['Location'].map_elements(geolocator.geocode)


In [4]:
fig = px.scatter_geo(df,
    lat='lat',
    lon='lon',
    hover_name='Quilt Name',
    hover_data={
        'Details': True,
        'lat': False,
        'lon': False,
    },
)
fig.update_geos(
    # fitbounds='locations',
    subunitcolor="#888888",
    # projection_type="orthographic"
)
fig.update_layout(
    geo_scope='usa',
    height=500,
    margin={"r":0,"t":0,"l":0,"b":0}
)

In [7]:
fig.write_html('graph.html')